In [ ]:
# Import packages - Modernized imports with type hints and additional libraries
import pandas as pd
import networkx as nx
import nx_parallel as nxp
from networkx.algorithms import community
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math
import json
import os
import hashlib
import datetime
from pyvis.network import Network
import time
from multiprocessing import Pool
from collections import defaultdict
from typing import Optional, Dict, List, Tuple

# Enable parallel backend globally for NetworkX
# Set this to the number of CPU cores you want to utilize.
# A safe default is often (os.cpu_count() or 4)
nx.config.backends.parallel.active = True
nx.config.backends.parallel.n_jobs = (os.cpu_count() or 4) - 1

# =================================== CENTRALIZED CONFIGURATION SYSTEM ===================================
# Comprehensive CONFIG dictionary structure for different analysis aspects
# This replaces all hardcoded parameters throughout the notebook

CONFIG = {
    # --- INPUT/OUTPUT CONFIGURATION ---
    "input_file": "followers_following.json",  # Path to input data file (JSON format preferred)
    "output_file_prefix": "Output/FollowWeb",   # Base path for all output files
    
    # --- PIPELINE CONFIGURATION ---
    "pipeline": {
        # Analysis strategy selection:
        # 1. "k-core": (Default) Prunes the full L1+L2 graph. Good for general overview.
        # 2. "reciprocal_k-core": Filters for mutuals (A follows B AND B follows A) then prunes. Good for finding "real" friend groups.
        # 3. "ego_alter_k-core": Creates a graph of only your L1 contacts, connected if they follow each other. Good for analyzing your immediate circle.
        "strategy": "k-core",    # Options: "k-core", "reciprocal_k-core", "ego_alter_k-core"
        
        # Skip computationally expensive structural analysis (community detection, centrality)
        "skip_analysis": False,  # Set to True to skip ALL computationally expensive structural analysis
        
        # Required for "ego_alter_k-core" strategy - the central node (you)
        "ego_username": "_alexs.life"  # Must be set if using "ego_alter_k-core"
    },

    # --- ANALYSIS CONFIGURATION ---
    "analysis": {
        # Specific username to find a path to (must be in your followers_following.json file)
        # Set to "" or None to disable manual path finding
        "contact_path_target": None,
    },

    # --- FAME ANALYSIS CONFIGURATION ---
    "fame_analysis": {
        # Find contact paths to every famous account identified
        "find_paths_to_all_famous": True, 
        
        # Minimum followers within your L1/L2 network for an account to be considered
        "min_followers_in_network": 5, 
        
        # Minimum ratio of (followers / following) to be considered famous
        # (e.g., 5.0 means 5 followers for every 1 person they follow)
        "min_fame_ratio": 5.0 
    },

    # --- PRUNING CONFIGURATION ---
    "pruning": {
        # Strategy-specific k-values (minimum connections required)
        # Nodes with fewer connections than this will be removed
        "k_values": {
            "k-core": 1,              # Conservative pruning for full network
            "reciprocal_k-core": 6,   # More aggressive pruning for mutual connections
            "ego_alter_k-core": 3,    # Moderate pruning for ego network
        },
        "default_k_value": 2  # Fallback if strategy name is incorrect
    },
    
    # --- VISUALIZATION CONFIGURATION ---
    "visualization": {
        # --- Shared Settings for Both HTML and PNG ---
        "node_size_metric": "degree",        # Options: "degree", "betweenness", "eigenvector"
        "base_node_size": 6,                 # Base size for nodes
        "node_size_multiplier": 5,           # Multiplier for node size scaling
        "scaling_algorithm": "logarithmic",  # Options: "logarithmic", "linear"
        
        "base_edge_width": 0.5,              # Base width for edges
        "edge_width_multiplier": 2,          # Multiplier for edge width scaling
        "edge_width_scaling": "logarithmic", # Options: "logarithmic", "linear"
        "intra_community_color": "#c0c0c0",  # Gray color for within-community edges
        "bridge_color": "#6e6e6e",           # Darker gray for between-community edges

        # --- Interactive HTML Visualization (Pyvis) Configuration ---
        "pyvis_interactive": { 
            "width": "100%",                 # Canvas width
            "height": "90vh",                # Canvas height
            "notebook": False,               # Set to True for Jupyter notebook display
            "show_labels": True,             # Set to False to hide node names for faster rendering
            "show_tooltips": True,           # Set to False to disable hover tooltips for faster loading
            "physics_solver": "forceAtlas2Based",  # Physics simulation algorithm
        },

        # --- Static PNG Image Configuration ---
        "static_image": {
            "generate": True,                # Set to True to generate static PNG images
            "layout": "spring",             # Options: "spring", "kamada_kawai", "circular", "shell"
            "with_labels": False,            # Labels are often too cluttered on static graphs
            "font_size": 8,                 # Font size for labels (if enabled)
            "image_size_inches": (25, 25),   # Image dimensions (width, height) in inches
            "dpi": 300,                     # Dots Per Inch - higher for better quality
            "spring_k": 0.3,               # Spring layout parameter (adjusts node repulsion)
            "spring_iterations": 50,        # Number of iterations for spring layout
            "edge_alpha": 0.3,              # Edge transparency (0.0 to 1.0)
            "node_alpha": 0.8,              # Node transparency (0.0 to 1.0)
            "edge_arrow_size": 8,            # Size of arrow heads on edges
            "show_legend": True              # Include legend in static images
        }
    }
}

# =================================== CONFIGURATION VALIDATION ===================================
# Comprehensive configuration validation with clear error messages
# This function validates all configuration parameters before execution

def validate_config(config: dict) -> bool:
    """
    Validates CONFIG before running pipeline.
    Raises exceptions for critical errors, prints warnings for minor issues.
    
    Args:
        config (dict): The configuration dictionary to validate
        
    Returns:
        bool: True if validation passes
        
    Raises:
        FileNotFoundError: If input file doesn't exist
        ValueError: If configuration parameters are invalid
    """
    print("=== VALIDATING CONFIGURATION ===")
    
    # --- INPUT FILE VALIDATION ---
    if not os.path.exists(config['input_file']):
        raise FileNotFoundError(f"Input file not found: {config['input_file']}. Please ensure the file exists.")
    print(f"âœ“ Input file exists: {config['input_file']}")
    
    # --- STRATEGY VALIDATION ---
    strategy = config['pipeline']['strategy']
    valid_strategies = ['k-core', 'reciprocal_k-core', 'ego_alter_k-core']
    if strategy not in valid_strategies:
        raise ValueError(f"Invalid strategy '{strategy}'. Must be one of: {valid_strategies}")
    print(f"âœ“ Strategy is valid: {strategy}")
    
    # --- EGO USERNAME VALIDATION (for ego_alter_k-core strategy) ---
    if strategy == "ego_alter_k-core":
        ego = config['pipeline']['ego_username']
        if not ego or ego == "_alexs.life":  # Check for placeholder value
            raise ValueError("'ego_username' must be set in CONFIG for 'ego_alter_k-core' strategy. Please replace the placeholder value.")
        print(f"âœ“ Ego username set for ego_alter_k-core: {ego}")
    
    # --- K-VALUES VALIDATION ---
    for strat, k_val in config['pruning']['k_values'].items():
        if not isinstance(k_val, int) or k_val < 0:
            raise ValueError(f"k-value for '{strat}' must be a non-negative integer: {k_val}")
    
    default_k = config['pruning']['default_k_value']
    if not isinstance(default_k, int) or default_k < 0:
        raise ValueError(f"default_k_value must be a non-negative integer: {default_k}")
    print(f"âœ“ K-values are valid: {config['pruning']['k_values']}")
    
    # --- FAME ANALYSIS VALIDATION ---
    min_followers = config['fame_analysis']['min_followers_in_network']
    if not isinstance(min_followers, int) or min_followers < 0:
        raise ValueError(f"min_followers_in_network must be a non-negative integer: {min_followers}")
    
    min_ratio = config['fame_analysis']['min_fame_ratio']
    if not isinstance(min_ratio, (int, float)) or min_ratio <= 0:
        raise ValueError(f"min_fame_ratio must be a positive number: {min_ratio}")
    print(f"âœ“ Fame analysis parameters are valid")
    
    # --- VISUALIZATION CONFIGURATION VALIDATION ---
    vis_config = config['visualization']
    
    # Validate node size metric
    valid_metrics = ['degree', 'betweenness', 'eigenvector']
    if vis_config['node_size_metric'] not in valid_metrics:
        raise ValueError(f"Invalid 'node_size_metric': {vis_config['node_size_metric']}. Must be one of: {valid_metrics}")
    
    # Validate scaling algorithms
    valid_scaling = ['logarithmic', 'linear']
    if vis_config['scaling_algorithm'] not in valid_scaling:
        raise ValueError(f"Invalid 'scaling_algorithm': {vis_config['scaling_algorithm']}. Must be one of: {valid_scaling}")
    
    if vis_config['edge_width_scaling'] not in valid_scaling:
        raise ValueError(f"Invalid 'edge_width_scaling': {vis_config['edge_width_scaling']}. Must be one of: {valid_scaling}")
    
    # Validate pyvis interactive configuration
    pyvis_config = vis_config.get('pyvis_interactive', {})
    required_pyvis_keys = ['width', 'height', 'physics_solver']
    missing_keys = [key for key in required_pyvis_keys if key not in pyvis_config]
    if missing_keys:
        raise ValueError(f"Missing required keys in 'visualization.pyvis_interactive': {missing_keys}")
    
    # Validate static image configuration
    static_config = vis_config.get('static_image', {})
    if static_config.get('generate', False):
        valid_layouts = ['spring', 'kamada_kawai', 'circular', 'shell']
        layout = static_config.get('layout', 'spring')
        if layout not in valid_layouts:
            raise ValueError(f"Invalid static image layout '{layout}'. Must be one of: {valid_layouts}")
        
        # Validate image dimensions
        image_size = static_config.get('image_size_inches', (25, 25))
        if not isinstance(image_size, (tuple, list)) or len(image_size) != 2:
            raise ValueError("image_size_inches must be a tuple/list of (width, height)")
        
        if any(not isinstance(dim, (int, float)) or dim <= 0 for dim in image_size):
            raise ValueError("image_size_inches dimensions must be positive numbers")
    
    print(f"âœ“ Visualization configuration is valid")
    
    # --- OUTPUT DIRECTORY VALIDATION ---
    output_dir = os.path.dirname(config['output_file_prefix'])
    if output_dir and not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"âœ“ Created output directory: {output_dir}")
        except OSError as e:
            raise ValueError(f"Cannot create output directory '{output_dir}': {e}")
    
    print("SUCCESS: Configuration validated successfully")
    print("=================================\n")
    return True

# Validate the configuration before proceeding
validate_config(CONFIG)
# Print configuration summary
print("=== CONFIGURATION LOADED ===")
print(f"Strategy: {CONFIG['pipeline']['strategy']}")
print(f"Input file: {CONFIG['input_file']}")
print(f"Output prefix: {CONFIG['output_file_prefix']}")
print(f"Skip analysis: {CONFIG['pipeline']['skip_analysis']}")
print(f"K-value for {CONFIG['pipeline']['strategy']}: {CONFIG['pruning']['k_values'].get(CONFIG['pipeline']['strategy'], CONFIG['pruning']['default_k_value'])}")
print("============================\n")

# =================================== DIRECTORY MANAGEMENT AND LOGGING ===================================
# Comprehensive directory management and logging throughout the analysis pipeline

def ensure_output_directory(path: str) -> bool:
    """
    Ensures that the output directory exists, creating it if necessary.
    
    Args:
        path (str): Path to the output file or directory
        
    Returns:
        bool: True if directory exists or was created successfully, False otherwise
    """
    try:
        # Extract directory from file path
        directory = os.path.dirname(path)
        
        # If no directory specified, use current directory
        if not directory:
            return True
        
        # Create directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
            print(f"Created output directory: {directory}")
        
        return True
        
    except OSError as e:
        print(f"ERROR: Could not create output directory for '{path}': {e}")
        return False
    except Exception as e:
        print(f"ERROR: Unexpected error creating directory for '{path}': {e}")
        return False

def log_analysis_step(step_name: str, details: str = "", timing: float = None) -> None:
    """
    Logs an analysis step with optional details and timing information.
    
    Args:
        step_name (str): Name of the analysis step
        details (str): Optional details about the step
        timing (float): Optional timing information in seconds
    """
    timestamp = datetime.datetime.now().strftime('%H:%M:%S')
    
    log_message = f"[{timestamp}] {step_name}"
    
    if details:
        log_message += f" - {details}"
    
    if timing is not None:
        log_message += f" (completed in {timing:.4f}s)"
    
    print(log_message)

def log_analysis_summary(G: nx.DiGraph, config: dict, timing_info: dict = None) -> None:
    """
    Logs a comprehensive summary of the analysis results.
    
    Args:
        G (nx.DiGraph): The analyzed graph
        config (dict): Configuration dictionary used for analysis
        timing_info (dict): Optional timing information for different analysis steps
    """
    print("\n" + "=" * 60)
    print("ANALYSIS SUMMARY")
    print("=" * 60)
    
    # Basic graph information
    print(f"Strategy: {config['pipeline']['strategy']}")
    print(f"Input File: {config['input_file']}")
    print(f"Final Graph: {G.number_of_nodes():,} nodes, {G.number_of_edges():,} edges")
    
    if G.number_of_nodes() > 0:
        density = nx.density(G)
        print(f"Network Density: {density:.6f}")
        
        # Community information
        communities_attr = nx.get_node_attributes(G, 'community')
        if communities_attr:
            num_communities = len(set(communities_attr.values()))
            print(f"Communities Detected: {num_communities}")
        else:
            print("Communities Detected: None (analysis skipped or failed)")
        
        # Degree statistics
        degrees = dict(G.degree())
        if degrees:
            degree_values = list(degrees.values())
            print(f"Average Degree: {sum(degree_values) / len(degree_values):.2f}")
            print(f"Degree Range: {min(degree_values)} - {max(degree_values)}")
    
    # Timing information
    if timing_info:
        print("\nPERFORMANCE BREAKDOWN:")
        total_time = sum(timing_info.values())
        for step_name, duration in timing_info.items():
            percentage = (duration / total_time * 100) if total_time > 0 else 0
            print(f"  {step_name:<25}: {duration:8.4f}s ({percentage:5.1f}%)")
        print(f"  {'TOTAL':<25}: {total_time:8.4f}s")
    
    print("=" * 60)

def create_progress_logger(total_steps: int, operation_name: str) -> callable:
    """
    Creates a progress logging function for multi-step operations.
    
    Args:
        total_steps (int): Total number of steps in the operation
        operation_name (str): Name of the operation being logged
        
    Returns:
        callable: Function to call for logging progress
    """
    start_time = time.perf_counter()
    
    def log_progress(current_step: int, step_description: str = ""):
        elapsed = time.perf_counter() - start_time
        percentage = (current_step / total_steps) * 100
        
        message = f"[{operation_name}] Step {current_step}/{total_steps} ({percentage:.1f}%)"
        if step_description:
            message += f" - {step_description}"
        message += f" [Elapsed: {elapsed:.2f}s]"
        
        print(message)
    
    return log_progress

# =================================== GRAPH LOADING AND STRATEGY FILTERING ===================================
# Advanced network analysis capabilities with strategy-specific filtering

def load_graph_from_json(filepath: str) -> nx.DiGraph:
    """
    Loads a directed graph from a JSON file with proper error handling.
    
    Expected format: A list of user objects
    [
     {
       "user": "username1",
       "followers": ["user2", "user3"],
       "following": ["user4", "user5"]
     },
     ...
    ]
    
    Args:
        filepath (str): Path to the JSON data file
        
    Returns:
        nx.DiGraph: Loaded graph or empty graph on error
    """
    G = nx.DiGraph()
    
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"ERROR: Invalid JSON format in {filepath}")
        print(f"       JSON error: {e}")
        return G
    except Exception as e:
        print(f"ERROR: Could not read file {filepath}: {e}")
        return G
    
    if not isinstance(data, list):
        print(f"ERROR: JSON root must be a LIST, got {type(data)}")
        return G
    
    print(f"Processing {len(data)} user entries...")
    
    # Process each user's data from the list
    for user_entry in data:
        if not isinstance(user_entry, dict):
            print(f"WARNING: Skipping item in list - data is not a dict")
            continue
        
        # Get username from the 'user' key
        username = user_entry.get('user')
        if not username:
            print("WARNING: Skipping item in list - 'user' key is missing or empty.")
            continue
        
        # Validate required keys
        if 'followers' not in user_entry or 'following' not in user_entry:
            print(f"WARNING: User '{username}' missing 'followers' or 'following' key")
            continue
        
        followers = user_entry.get('followers', [])
        following = user_entry.get('following', [])
        
        if not isinstance(followers, list):
            print(f"WARNING: User '{username}' - 'followers' is not a list")
            followers = []
        if not isinstance(following, list):
            print(f"WARNING: User '{username}' - 'following' is not a list")
            following = []
        
        # Add edges
        for follower in followers:
            if follower:  # Skip empty strings
                G.add_edge(follower, username)  # Follower -> User
        
        for followee in following:
            if followee:  # Skip empty strings
                G.add_edge(username, followee)  # User -> Followee
    
    print(f"Initial graph loaded: {G.number_of_nodes():,} nodes, {G.number_of_edges():,} edges.")
    return G

def filter_by_reciprocity(G: nx.DiGraph) -> nx.DiGraph:
    """
    Creates a new graph containing only reciprocal edges (mutual followers).
    
    Args:
        G (nx.DiGraph): Input graph
        
    Returns:
        nx.DiGraph: New graph with only mutual connections, or empty graph if none exist
    """
    G_reciprocal = nx.DiGraph()
    
    # Keep only edges where the reverse edge also exists
    reciprocal_edges = [edge for edge in G.edges() if G.has_edge(edge[1], edge[0])]
    G_reciprocal.add_edges_from(reciprocal_edges)

    # Remove nodes that now have 0 degree
    G_reciprocal.remove_nodes_from(list(nx.isolates(G_reciprocal)))

    print(f"Filtered for mutuals: {G_reciprocal.number_of_nodes():,} nodes, {G_reciprocal.number_of_edges():,} edges.")
    return G_reciprocal

def create_ego_alter_graph(G: nx.DiGraph, ego_username: str) -> nx.DiGraph:
    """
    Creates an "alter graph" showing connections between the ego's L1 contacts.
    
    Args:
        G (nx.DiGraph): Input graph
        ego_username (str): The central node (ego)
        
    Returns:
        nx.DiGraph: Graph of L1 contacts and their connections, or empty graph on error
    """
    if ego_username not in G:
        print(f"ERROR: Ego node '{ego_username}' not in graph. Check CONFIG.")
        return nx.DiGraph()

    # 1. Identify Alters (L1 nodes)
    try:
        followers = set(G.predecessors(ego_username))
    except nx.NetworkXError:
        followers = set()
        
    try:
        following = set(G.successors(ego_username))
    except nx.NetworkXError:
        following = set()
        
    alters = followers.union(following)

    if not alters:
        print("WARNING: No alters (L1 connections) found for this ego.")
        return nx.DiGraph()

    # 2. Create a new graph containing only connections between alters
    alter_graph = G.subgraph(alters).copy()
    
    # Remove isolates (alters who don't connect to any other alters)
    alter_graph.remove_nodes_from(list(nx.isolates(alter_graph)))

    print(f"Alter graph created: {alter_graph.number_of_nodes():,} alters, {alter_graph.number_of_edges():,} connections between them.")
    return alter_graph

def prune_graph(G: nx.DiGraph, min_degree: int) -> nx.DiGraph:
    """
    Uses nx.k_core function to find the maximal subgraph 
    where all nodes have degree >= min_degree.
    
    Args:
        G (nx.DiGraph): Input graph
        min_degree (int): Minimum degree threshold (k-value)
        
    Returns:
        nx.DiGraph: Pruned graph (k-core subgraph)
    """
    if min_degree <= 0:
        print("Pruning skipped (min_degree <= 0).")
        return G

    # nx.k_core finds the subgraph where all nodes have at least k-degree
    # For DiGraphs, .degree() is in+out, which matches your original logic.
    G_pruned = nx.k_core(G, k=min_degree)

    nodes_removed = G.number_of_nodes() - G_pruned.number_of_nodes()
    
    print(f"Pruning complete. Removed {nodes_removed:,} nodes.")
    print(f"Final pruned graph: {G_pruned.number_of_nodes():,} nodes, {G_pruned.number_of_edges():,} edges.")
    return G_pruned

def apply_strategy_filtering(G: nx.DiGraph, strategy: str, ego_username: Optional[str] = None) -> nx.DiGraph:
    """
    Applies strategy-specific graph filtering for different analysis approaches.
    
    Args:
        G (nx.DiGraph): Input graph
        strategy (str): Analysis strategy ("k-core", "reciprocal_k-core", "ego_alter_k-core")
        ego_username (Optional[str]): Required for "ego_alter_k-core" strategy
        
    Returns:
        nx.DiGraph: Filtered graph based on strategy
    """
    print(f"Applying '{strategy}' strategy filtering...")
    
    if strategy == "k-core":
        # Use the full graph as-is
        return G
    elif strategy == "reciprocal_k-core":
        # Filter for mutual connections only
        return filter_by_reciprocity(G)
    elif strategy == "ego_alter_k-core":
        # Create ego-alter network
        if not ego_username:
            print("ERROR: ego_username required for 'ego_alter_k-core' strategy")
            return nx.DiGraph()
        return create_ego_alter_graph(G, ego_username)
    else:
        print(f"WARNING: Unknown strategy '{strategy}'. Using full graph.")
        return G

# =================================== LOAD AND PROCESS GRAPH ===================================
# Load graph from JSON data file with strategy-specific filtering

# Initialize timing tracking
pipeline_start_time = time.perf_counter()
timing_info = {}

print("=== LOADING GRAPH FROM DATA FILE ===")
log_analysis_step("Pipeline Started", f"Strategy: {CONFIG['pipeline']['strategy']}")

# Ensure output directory exists
ensure_output_directory(CONFIG['output_file_prefix'])

graph_load_start = time.perf_counter()
DATA_FILE = CONFIG['input_file']

# Load the initial graph from JSON
G = load_graph_from_json(DATA_FILE)

if G.number_of_nodes() == 0:
    print("ERROR: No graph data loaded. Exiting.")
    exit()

# Apply strategy-specific filtering
strategy = CONFIG['pipeline']['strategy']
ego_username = CONFIG['pipeline'].get('ego_username')

G_filtered = apply_strategy_filtering(G, strategy, ego_username)

if G_filtered.number_of_nodes() == 0:
    print("WARNING: Strategy filtering resulted in empty graph.")
    G_filtered = G  # Fall back to original graph

# Apply k-core pruning based on strategy
k_value = CONFIG['pruning']['k_values'].get(strategy, CONFIG['pruning']['default_k_value'])
print(f"Applying k-core pruning with k={k_value}...")
G_final = prune_graph(G_filtered, k_value)

print(f"Final processed graph: {G_final.number_of_nodes():,} nodes, {G_final.number_of_edges():,} edges.")

# Record graph loading timing
graph_load_end = time.perf_counter()
timing_info['Graph Loading'] = graph_load_end - graph_load_start
log_analysis_step("Graph Loading Complete", 
                 f"{G_final.number_of_nodes():,} nodes, {G_final.number_of_edges():,} edges", 
                 timing_info['Graph Loading'])

print("=====================================\n")

# Update the main graph variable for downstream processing
G = G_final

# =================================== PROGRESS TRACKER CLASS ===================================
# Comprehensive progress tracking for long-running operations

class ProgressTracker:
    """
    A utility class to print progress updates for long-running loops.
    
    Usage:
        tracker = ProgressTracker(total=1000, title="Processing items")
        for i in range(1000):
            tracker.update(i + 1)
        tracker.complete()
        
    For chunked tasks, set num_updates to number of chunks:
        tracker = ProgressTracker(total=50, title="Processing", num_updates=50)
        for i in range(0, 5000, 100):  # 50 chunks of 100 items
            # ... do work ...
            tracker.update((i // 100) + 1)  # Update with chunk number
    """
    
    @staticmethod
    def _format_time(seconds: float) -> str:
        """Convert seconds to human-readable format."""
        if seconds < 60.0:
            return f"{seconds:.1f}s"
        minutes = int(seconds // 60)
        remaining_sec = int(seconds % 60)
        if minutes < 60:
            return f"{minutes}m {remaining_sec}s"
        hours = minutes // 60
        remaining_min = minutes % 60
        return f"{hours}h {remaining_min}m"

    def __init__(
        self,
        total: int,
        title: str,
        num_updates: int = 10,
        threshold_sec: float = 3.0,
        show_percentage: bool = True,
        show_count: bool = True,
        bar_width: int = 30
    ):
        """
        Initialize the progress tracker.
        
        Args:
            total: Total number of items/chunks to process
            title: Title to display
            num_updates: Target number of progress updates (usually = num chunks)
            threshold_sec: Only show progress if total task estimated to exceed this (seconds)
            show_percentage: Include percentage in output
            show_count: Include item count in output
            bar_width: Width of progress bar in characters (0 to disable)
        """
        self.total = max(1, total)
        self.title = title
        self.num_updates = max(1, num_updates)
        self.threshold_sec = max(0.1, threshold_sec)
        self.show_percentage = show_percentage
        self.show_count = show_count
        self.bar_width = max(0, bar_width)
        self.show_progress = False

        self.title_printed = False 
        self.decision_made = False 
        self.update_every_n = max(1, self.total // self.num_updates)
        
        self.start_time = time.perf_counter()
        self.last_printed_item = -1
        
        # For time estimation
        self.first_chunk_time = None  # Time taken for first chunk
        self.ema_rate = None  # Exponential moving average of processing rate
        self.ema_alpha = 0.3  # Smoothing factor

        # Print the title and an initial "0%" line immediately.
        print(f"{self.title}...")
        self.title_printed = True
        
        progress_str = self._format_progress_line(
            current_item=0,
            percent=0.0,
            elapsed=0.0,
            remaining_time=0.0  # 0.0 remaining_time triggers our fallback
        )
        print(progress_str, end="\r", flush=True)
        self.last_printed_item = 0 # Mark item 0 as "printed"
    
    def __enter__(self):
        """Context manager entry."""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        self.complete()
        return False
    
    def _make_decision(self, current_item: int) -> None:
        """
        After first chunk, estimate total time and decide whether to show progress.
        """
        if self.decision_made or current_item == 0:
            return
        
        elapsed = time.perf_counter() - self.start_time
        self.first_chunk_time = elapsed
        
        # Estimate total time based on first chunk
        if current_item > 0:
            chunks_remaining = self.total - current_item
            estimated_total_time = elapsed + (chunks_remaining * (elapsed / current_item))
        else:
            estimated_total_time = 0
        
        # Decide: show progress only if estimated total exceeds threshold
        self.show_progress = estimated_total_time > self.threshold_sec
        self.decision_made = True
        
        if self.show_progress and not self.title_printed:
            print(f"{self.title}...")
            self.title_printed = True
    
    def _estimate_remaining_time(self, current_item: int) -> float:
        """
        Estimate remaining time using exponential moving average of rate.
        """
        if current_item <= 0:
            return 0.0
        
        elapsed = time.perf_counter() - self.start_time
        
        if elapsed <= 0:
            return 0.0
        
        # Calculate rate for this item
        current_rate = current_item / elapsed
        
        # Update EMA
        if self.ema_rate is None:
            self.ema_rate = current_rate
        else:
            self.ema_rate = (self.ema_alpha * current_rate + 
                            (1 - self.ema_alpha) * self.ema_rate)
        
        # Estimate remaining time
        remaining_items = self.total - current_item
        if self.ema_rate > 0:
            return remaining_items / self.ema_rate
        return 0.0
    
    def _create_progress_bar(self, percent: float) -> str:
        """
        Generate ASCII progress bar.
        """
        if self.bar_width <= 0:
            return ""
        
        # Calculate how many full '█' characters to show
        filled_len = int(self.bar_width * percent / 100.0)
        
        # Create the bar string
        bar_fill = '█' * filled_len
        
        # Pad the rest with spaces to match the total bar_width
        bar = bar_fill.ljust(self.bar_width, ' ')
        
        return f" [{bar}]"
    
    def _format_progress_line(
        self,
        current_item: int,
        percent: float,
        elapsed: float,
        remaining_time: float
    ) -> str:
        """Format the progress line with all requested information."""
        parts = []
        
        # Count information
        if self.show_count:
            parts.append(f"{current_item}/{self.total}")
        
        # Percentage
        if self.show_percentage:
            parts.append(f"({percent:.1f}%)")
        
        # Progress bar
        bar = self._create_progress_bar(percent)
        if bar:
            parts.append(bar)
        
        # Time information
        if percent < 100.0 and percent > 1.0 and current_item < self.total:
            if remaining_time > 0:
                remaining_str = self._format_time(remaining_time)
                parts.append(f"- Est. {remaining_str} remaining")
            else:
                parts.append("- Processing...") 
        elif current_item >= self.total:
            elapsed_str = self._format_time(elapsed)
            parts.append(f"- Completed in {elapsed_str}")
        
        return "    Progress: " + " ".join(parts)
    
    def update(self, current_item: int) -> None:
        """
        Call this inside the loop with the current item count (1-based).
        
        Args:
            current_item: Current iteration number (1-based) or chunk number
        """
        is_last_item = (current_item >= self.total)
        is_update_step = (current_item % self.update_every_n == 0)

        # Decide whether to show progress
        if not self.decision_made:
            if (is_update_step and current_item > 0) or is_last_item:
                self._make_decision(current_item)
        
        if not self.show_progress:
            return  # Hides all output for short tasks

        if (is_update_step or is_last_item) and current_item > self.last_printed_item:
            
            self.last_printed_item = current_item
            current_item = min(current_item, self.total)
            
            elapsed = time.perf_counter() - self.start_time
            percent = (current_item / self.total) * 100.0
            remaining_time = self._estimate_remaining_time(current_item)
            
            # Format and print progress
            progress_str = self._format_progress_line(
                current_item,
                percent,
                elapsed,
                remaining_time
            )
            print(progress_str, flush=True)
    
    def complete(self) -> None:
        """Call after the loop finishes. Prints completion info."""
        
        if self.show_progress:
            # Print final 100% line
            self.update(self.total)
    
    def reset(self, total: Optional[int] = None) -> None:
        """Reset the tracker to start a new task (useful for reuse)."""
        if total is not None:
            self.total = max(1, total)
        
        self.start_time = time.perf_counter()
        self.last_printed_item = -1 
        self.ema_rate = None
        self.show_progress = False
        self.title_printed = False
        self.decision_made = False
        self.first_chunk_time = None

# =================================== NETWORK ANALYSIS AND COMMUNITY DETECTION ===================================
# Advanced network analysis with community detection and centrality calculations

def analyze_network(G: nx.DiGraph) -> nx.DiGraph:
    """
    Calculates network metrics and attaches them as node attributes.
    Performs community detection and centrality calculations.
    
    Args:
        G (nx.DiGraph): Input graph
        
    Returns:
        nx.DiGraph: Graph with analysis results stored as node attributes
    """
    MIN_NODES_FOR_ANALYSIS = 2
    
    if G.number_of_nodes() < MIN_NODES_FOR_ANALYSIS:
        print(f"WARNING: Graph has fewer than {MIN_NODES_FOR_ANALYSIS} nodes. Skipping analysis.")
        return G
    
    # This task has 3 main steps: 1. Louvain, 2. Betweenness, 3. Eigenvector
    # We set total=3 and update after each step.
    # We also give it a short threshold_sec (e.g., 1.0s) so it appears
    # even if the first step (Louvain) is fast.
    tracker_title = "Analyzing structure (Communities, Centrality)"
    tracker = ProgressTracker(
        total=3, 
        title=tracker_title, 
        num_updates=3, 
        threshold_sec=1.0, 
        bar_width=30
    )
    
    # --- 1. Community Detection (Louvain) ---
    G_undirected = G.to_undirected()
    
    if G_undirected.number_of_edges() == 0:
        print("WARNING: Graph has no edges. Skipping Louvain community detection.")
        communities = []
    else:
        communities = community.louvain_communities(G_undirected, seed=123)
    
    print(f"Detected {len(communities)} communities (Louvain).")
    partition = {node: i for i, comm in enumerate(communities) for node in comm}
    nx.set_node_attributes(G, partition, 'community')
    
    tracker.update(1)  # Step 1 complete

    # --- 2. Centrality Measures ---
    degree_dict = dict(G.degree())
    
    try:
        k_sample = int(math.sqrt(G.number_of_nodes()))
        betweenness_dict = nx.betweenness_centrality(G, k=k_sample, seed=123)
    except Exception as e:
        print(f"WARNING: Could not calculate betweenness centrality ({e}). Defaulting to 0.")
        betweenness_dict = {node: 0 for node in G.nodes()}

    tracker.update(2)  # Step 2 complete

    try:
        eigenvector_dict = nx.eigenvector_centrality(G, max_iter=1000, nstart={n:1 for n in G.nodes()})
    except Exception as e:
        print(f"WARNING: Could not calculate eigenvector centrality ({e}). Defaulting to 0.")
        eigenvector_dict = {node: 0 for node in G.nodes()}

    tracker.update(3)  # Step 3 complete
    tracker.complete()  # All steps done

    # Store all metrics as node attributes
    nx.set_node_attributes(G, degree_dict, 'degree')
    nx.set_node_attributes(G, betweenness_dict, 'betweenness')
    nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
    
    print(f"Network analysis complete. Communities: {len(communities)}, Nodes analyzed: {G.number_of_nodes():,}")
    return G

# =================================== PERFORM NETWORK ANALYSIS ===================================
# Apply community detection and centrality analysis if not skipped

if not CONFIG['pipeline']['skip_analysis']:
    print("=== PERFORMING NETWORK ANALYSIS ===")
    network_analysis_start = time.perf_counter()
    log_analysis_step("Network Analysis Started", "Community detection and centrality calculations")
    
    G = analyze_network(G)
    
    network_analysis_end = time.perf_counter()
    timing_info['Network Analysis'] = network_analysis_end - network_analysis_start
    log_analysis_step("Network Analysis Complete", 
                     f"Communities and centrality calculated", 
                     timing_info['Network Analysis'])
    
    print("Network analysis completed.")
    print("====================================\n")
else:
    print("=== SKIPPING NETWORK ANALYSIS (as configured) ===")
    log_analysis_step("Network Analysis Skipped", "Using default attributes for visualization")
    
    # Set default attributes for visualization compatibility
    degree_dict = dict(G.degree())
    nx.set_node_attributes(G, degree_dict, 'degree')
    nx.set_node_attributes(G, {n: 0 for n in G.nodes()}, 'community')
    nx.set_node_attributes(G, {n: 0.0 for n in G.nodes()}, 'betweenness')
    nx.set_node_attributes(G, {n: 0.0 for n in G.nodes()}, 'eigenvector')
    
    timing_info['Network Analysis'] = 0.0
    print("Default attributes set for visualization compatibility.")
    print("====================================================\n")

# =================================== FAME ANALYSIS AND PATH FINDING ===================================
# Advanced fame analysis with configurable thresholds and path finding capabilities

def find_famous_accounts(G: nx.DiGraph, min_followers: int, min_ratio: float) -> Tuple[List[Dict], List[Dict]]:
    """
    Analyzes the full graph to find accounts that match the "famous" heuristic:
    - High followers (in-degree)
    - Low following (out-degree)
    - High ratio of followers-to-following
    
    Args:
        G (nx.DiGraph): Input graph
        min_followers (int): Minimum followers within network
        min_ratio (float): Minimum followers/following ratio
    
    Returns:
        Tuple[List[Dict], List[Dict]]: (unreachable_famous, reachable_famous)
                                     - unreachable_famous: List of famous accounts with following_in_network=0
                                     - reachable_famous: List of famous accounts with following_in_network>=1
    """
    unreachable_famous = []
    reachable_famous = []
    
    if G.number_of_nodes() == 0:
        return unreachable_famous, reachable_famous
    
    # Get all degrees at once in optimized bulk operations
    print("       Caching all in-degrees and out-degrees...")
    in_degree_dict = dict(G.in_degree())
    out_degree_dict = dict(G.out_degree())

    # Use .items() to iterate over nodes and their pre-calculated in-degree
    for node, in_deg in in_degree_dict.items():
        # 1. Filter out nodes that aren't popular enough in your network
        if in_deg < min_followers:
            continue
        
        # Get the pre-calculated out-degree
        out_deg = out_degree_dict.get(node, 0)  # .get() is safer

        # 2. Calculate fame ratio (handle division by zero)
        ratio = float('inf')  # Highest possible ratio
        if out_deg > 0:
            ratio = in_deg / out_deg
        
        # 3. Filter by ratio
        if ratio >= min_ratio:
            account_info = {
                "username": node,
                "followers_in_network": in_deg,
                "following_in_network": out_deg,
                "ratio": ratio
            }
            
            # Separate into unreachable (follows nobody) and reachable
            if out_deg == 0:
                unreachable_famous.append(account_info)
            else:
                reachable_famous.append(account_info)
    
    # Sort by ratio (descending), then by followers (descending)
    unreachable_famous.sort(key=lambda x: (x['ratio'], x['followers_in_network']), reverse=True)
    reachable_famous.sort(key=lambda x: (x['ratio'], x['followers_in_network']), reverse=True)
    
    return unreachable_famous, reachable_famous

def get_contact_path(G: nx.DiGraph, ego_username: str, target_username: str) -> Optional[List[str]]:
    """
    Silently finds the shortest path from target to ego.
    
    Args:
        G (nx.DiGraph): Input graph
        ego_username (str): The ego node (destination)
        target_username (str): The target node (source)
    
    Returns:
        Optional[List[str]]: The path (a list of nodes) if found, None otherwise
    """
    if ego_username not in G or target_username not in G:
        return None
    
    try:
        path = nx.shortest_path(G, source=target_username, target=ego_username)
        return path
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return None
    except Exception as e:
        print(f"ERROR: Unexpected error in get_contact_path ({target_username}): {e}")
        return None
        
def print_detailed_contact_path(G: nx.DiGraph, ego_username: str, target_username: str) -> bool:
    """
    Finds and prints a detailed "chain of influence" from a target user back to the ego.
    This is a verbose function for a single manual target.
    
    Args:
        G (nx.DiGraph): Input graph
        ego_username (str): The ego node (destination)
        target_username (str): The target node (source)
    
    Returns:
        bool: True if a path was found, False otherwise
    """
    print(f"\n---- Finding Path: {target_username} -> {ego_username} ----")

    if ego_username not in G:
        print(f"ERROR: Ego node '{ego_username}' not in the *full* graph.")
        return False
    if target_username not in G:
        print(f"ERROR: Target node '{target_username}' not in the *full* graph.")
        print("       This person might not be in your L1/L2 network, or you may have a typo.")
        return False

    path = get_contact_path(G, ego_username, target_username)

    if path:
        print("\n*** SUCCESS: Contact path found! ***")
        
        print("       Follows Chain (Target to Ego):")
        for i in range(len(path) - 1):
            print(f"         {path[i]:<20} -> follows -> {path[i+1]}")
            
        print(f"       Path length: {len(path) - 1} step(s).")
        
        print("       Action Plan (Read from bottom up):")
        print(f"         1. You contact:         {path[-2]}")
        
        action_step = 2
        for i in range(len(path) - 2, 0, -1):
            print(f"         {action_step}. {path[i]} contacts: {path[i-1]}")
            action_step += 1
            
        print(f"         ...who can contact '{path[0]}'.")
        return True
    else:
        print(f"\n*** NO PATH found from '{target_username}' to '{ego_username}'. ***")
        print("       No 'chain of influence' exists in your network.")
        return False

def perform_fame_analysis(G: nx.DiGraph, config: dict) -> None:
    """
    Performs comprehensive fame analysis and path finding based on configuration.
    
    Args:
        G (nx.DiGraph): Input graph
        config (dict): Configuration dictionary
    """
    print("=== PERFORMING FAME ANALYSIS ===")
    
    # Get fame analysis parameters
    min_followers = config['fame_analysis']['min_followers_in_network']
    min_ratio = config['fame_analysis']['min_fame_ratio']
    find_paths_to_all = config['fame_analysis']['find_paths_to_all_famous']
    
    print(f"Fame criteria: min {min_followers} followers, min {min_ratio:.1f}x ratio")
    
    # Find famous accounts
    unreachable_famous, reachable_famous = find_famous_accounts(G, min_followers, min_ratio)
    
    total_famous = len(unreachable_famous) + len(reachable_famous)
    print(f"Found {total_famous} famous accounts ({len(reachable_famous)} reachable, {len(unreachable_famous)} unreachable)")
    
    if total_famous == 0:
        print("No famous accounts found with current criteria.")
        return
    
    # Display top famous accounts
    print("\n--- TOP FAMOUS ACCOUNTS (REACHABLE) ---")
    for i, account in enumerate(reachable_famous[:10]):  # Show top 10
        ratio_str = "∞" if account['ratio'] == float('inf') else f"{account['ratio']:.1f}"
        print(f"  {i+1:2d}. {account['username']:<20} | {account['followers_in_network']:4d} followers | {account['following_in_network']:3d} following | {ratio_str}x ratio")
    
    if unreachable_famous:
        print("\n--- TOP FAMOUS ACCOUNTS (UNREACHABLE) ---")
        for i, account in enumerate(unreachable_famous[:5]):  # Show top 5
            print(f"  {i+1:2d}. {account['username']:<20} | {account['followers_in_network']:4d} followers | 0 following | ∞x ratio")
    
    # Manual path finding for specific target
    manual_target = config['analysis'].get('contact_path_target')
    if manual_target:
        ego_username = config['pipeline'].get('ego_username', '_alexs.life')
        print_detailed_contact_path(G, ego_username, manual_target)
    
    # Path finding for all famous accounts
    if find_paths_to_all and reachable_famous:
        ego_username = config['pipeline'].get('ego_username', '_alexs.life')
        print(f"\n--- CONTACT PATHS TO FAMOUS ACCOUNTS ---")
        
        paths_found = 0
        for account in reachable_famous[:20]:  # Limit to top 20 for performance
            path = get_contact_path(G, ego_username, account['username'])
            if path:
                paths_found += 1
                path_length = len(path) - 1
                print(f"  {account['username']:<20} | {path_length} steps | Path: {' -> '.join(path[:3])}{'...' if len(path) > 3 else ''}")
        
        print(f"\nFound paths to {paths_found}/{len(reachable_famous[:20])} famous accounts.")
    
    print("Fame analysis completed.")
    print("=================================\n")

# =================================== PERFORM FAME ANALYSIS ===================================
# Execute fame analysis and path finding based on configuration

if G.number_of_nodes() > 0:
    fame_analysis_start = time.perf_counter()
    log_analysis_step("Fame Analysis Started", "Identifying influential accounts and paths")
    
    perform_fame_analysis(G, CONFIG)
    
    fame_analysis_end = time.perf_counter()
    timing_info['Fame Analysis'] = fame_analysis_end - fame_analysis_start
    log_analysis_step("Fame Analysis Complete", 
                     "Famous accounts identified and paths calculated", 
                     timing_info['Fame Analysis'])
else:
    print("=== SKIPPING FAME ANALYSIS (empty graph) ===")
    log_analysis_step("Fame Analysis Skipped", "Empty graph")
    timing_info['Fame Analysis'] = 0.0
    print("============================================\n")

# =================================== UTILITY FUNCTIONS FOR VISUALIZATION ===================================
# Helper functions for filename generation, color mapping, and scaling algorithms

def generate_output_filename(prefix: str, strategy: str, k_value: int, extension: str) -> str:
    """
    Generates a descriptive and unique output filename based on config and time.
    Format: {prefix}-{strategy}-k{k_value}-{config_hash}.{extension}
    
    Args:
        prefix (str): Base filename prefix
        strategy (str): Analysis strategy name
        k_value (int): K-value used for pruning
        extension (str): File extension (html, png, txt)
        
    Returns:
        str: Unique filename with timestamp hash
    """
    # Get current timestamp to ensure uniqueness
    timestamp = datetime.datetime.now().isoformat()
    
    # Create a short hash of relevant config and time to avoid collisions
    config_str = f"{strategy}-{k_value}-{timestamp}"
    config_hash = hashlib.sha256(config_str.encode()).hexdigest()[:6]
    
    return f"{prefix}-{strategy}-k{k_value}-{config_hash}.{extension}"

def get_community_colors(num_communities: int) -> dict:
    """
    Generates a color map for communities.
    Returns a dict mapping community_id -> hex color string and RGBA tuple.
    
    Args:
        num_communities (int): Number of communities to generate colors for
        
    Returns:
        dict: Dictionary with 'hex' and 'rgba' color mappings
    """
    if num_communities > 0:
        palette = plt.colormaps.get_cmap('viridis').resampled(num_communities)
        colors = palette(range(num_communities))
        return {
            'hex': {i: f'#{int(c[0]*255):02x}{int(c[1]*255):02x}{int(c[2]*255):02x}' 
                    for i, c in enumerate(colors)},
            'rgba': {i: c for i, c in enumerate(colors)}
        }
    return {'hex': {0: '#808080'}, 'rgba': {0: (0.5, 0.5, 0.5, 1.0)}}

def _get_scaled_size(value: float, base_size: float, multiplier: float, algorithm: str) -> float:
    """
    Helper to calculate node/edge size based on a metric's value.
    
    Args:
        value (float): The metric value to scale
        base_size (float): Base size before scaling
        multiplier (float): Scaling multiplier
        algorithm (str): Scaling algorithm ('logarithmic' or 'linear')
        
    Returns:
        float: Scaled size value
    """
    if algorithm == 'logarithmic':
        # Use log1p (log(1+x)) to handle 0 values gracefully
        return base_size + math.log1p(value) * multiplier
    else:  # linear
        return base_size + value * multiplier

# =================================== NODE AND EDGE METRIC CALCULATIONS ===================================
# Advanced metric calculations for both HTML and PNG visualizations

def calculate_edge_metrics(G: nx.DiGraph, vis_config: dict) -> dict:
    """
    Calculates edge metrics for both HTML and PNG visualizations.
    Returns a dict mapping (u, v) -> {'width': float, 'color': str, 'is_mutual': bool, 'is_bridge': bool, 'common_neighbors': int}
    
    This function iterates over the undirected graph's edges to avoid processing mutual pairs twice.
    It calculates shared metrics (like common_neighbors) once per undirected pair
    and then adds the correct directed edge entry/entries to the dictionary.
    
    Args:
        G (nx.DiGraph): Input graph with community attributes
        vis_config (dict): Visualization configuration
        
    Returns:
        dict: Edge metrics mapping (u, v) -> metrics dict
    """
    print("Calculating shared edge metrics (embeddedness, bridging, reciprocity)...")
    
    G_undirected = G.to_undirected()
    edge_metrics = {}
    
    base_edge_width = vis_config.get("base_edge_width", 0.5)
    edge_width_multiplier = vis_config.get("edge_width_multiplier", 1.5)
    edge_width_scaling = vis_config.get("edge_width_scaling", "logarithmic")
    bridge_color = vis_config.get("bridge_color", "#6e6e6e")
    
    # Get community color map
    num_communities = len(set(nx.get_node_attributes(G, 'community').values()))
    color_maps = get_community_colors(num_communities)
    color_map_hex = color_maps['hex']
    
    # Iterate over each UNDIRECTED edge pair once
    for u, v in G_undirected.edges():
        
        # --- 1. Calculate metrics shared by the pair (u, v) ---
        
        # Calculate common neighbors (embeddedness)
        try:
            # Use sum(1 for...) to efficiently get iterator length without list()
            common_neighbors_iter = nx.common_neighbors(G_undirected, u, v)
            num_common = sum(1 for _ in common_neighbors_iter)
        except nx.NetworkXError:
            num_common = 0
        
        # Calculate edge width
        edge_width = _get_scaled_size(
            num_common,
            base_edge_width,
            edge_width_multiplier,
            edge_width_scaling
        )
        
        # Determine if bridge edge
        u_comm = G.nodes[u].get('community', -1)
        v_comm = G.nodes[v].get('community', -2)
        is_bridge = (u_comm != v_comm)
        
        # Set edge color
        if is_bridge:
            edge_color = bridge_color
        else:
            edge_color = color_map_hex.get(u_comm, vis_config.get("intra_community_color", "#c0c0c0"))

        # --- 2. Check directionality in G and add to metrics dict ---
            
        has_u_v = G.has_edge(u, v)
        has_v_u = G.has_edge(v, u)
        is_mutual = has_u_v and has_v_u

        # Define the base metrics for this pair
        base_metrics = {
            'width': edge_width,
            'color': edge_color,
            'is_bridge': is_bridge,
            'common_neighbors': num_common,
        }

        if is_mutual:
            # Add ONE entry for the mutual pair, key (u, v) by convention.
            # Downstream functions know to render this as a mutual arrow.
            edge_metrics[(u, v)] = {
                **base_metrics,
                'is_mutual': True,
                'u_comm': u_comm, # 'from' node is u
                'v_comm': v_comm  # 'to' node is v
            }
        
        else:
            # Add entries for each one-way edge that exists
            if has_u_v:
                edge_metrics[(u, v)] = {
                    **base_metrics,
                    'is_mutual': False,
                    'u_comm': u_comm, # 'from' node is u
                    'v_comm': v_comm  # 'to' node is v
                }
            
            if has_v_u:
                edge_metrics[(v, u)] = {
                    **base_metrics,
                    'is_mutual': False,
                    'u_comm': v_comm, # 'from' node is v
                    'v_comm': u_comm  # 'to' node is u
                }
                
    return edge_metrics

def calculate_node_metrics(G: nx.DiGraph, vis_config: dict) -> dict:
    """
    Calculates node size and color information for both HTML and PNG visualizations.
    Returns a dict mapping node -> {'size': float, 'community': int, 'color_hex': str, 'color_rgba': tuple}
    
    Args:
        G (nx.DiGraph): Input graph with analysis attributes
        vis_config (dict): Visualization configuration
        
    Returns:
        dict: Node metrics mapping node -> metrics dict
    """
    
    # Handle the case where analysis was skipped (attributes are missing)
    if not nx.get_node_attributes(G, 'community'):
        print("Using default metrics (degree, community 0) as analysis was skipped.")
        node_metrics = {}
        # Set all community IDs to 0
        nx.set_node_attributes(G, {n: 0 for n in G.nodes()}, 'community') 
        
        # Set all centrality/degree to 0 or 1 for visualization fallback
        for n in G.nodes():
            G.nodes[n]['degree'] = G.degree(n) # Use actual degree for sizing fallback
            G.nodes[n]['betweenness'] = 0.0
            G.nodes[n]['eigenvector'] = 0.0
            
    # Regular metric calculation
    node_metrics = {}
    size_metric = vis_config['node_size_metric']
    base_size = vis_config['base_node_size']
    multiplier = vis_config['node_size_multiplier']
    scaling_alg = vis_config['scaling_algorithm']
    
    # Get community colors
    num_communities = len(set(nx.get_node_attributes(G, 'community').values()))
    color_maps = get_community_colors(num_communities)
    
    for node, attrs in G.nodes(data=True):
        # Use actual metric if available, otherwise fallback to degree
        metric_value = attrs.get(size_metric, attrs.get('degree', 1))
        community_id = attrs.get('community', 0)
        
        node_size = _get_scaled_size(
            metric_value,
            base_size,
            multiplier,
            scaling_alg
        )
        
        node_metrics[node] = {
            'size': node_size,
            'community': community_id,
            'color_hex': color_maps['hex'].get(community_id, '#808080'),
            'color_rgba': color_maps['rgba'].get(community_id, (0.5, 0.5, 0.5, 1.0)),
            'degree': attrs.get('degree', 0),
            'betweenness': attrs.get('betweenness', 0),
            'eigenvector': attrs.get('eigenvector', 0)
        }
    
    return node_metrics

# =================================== INTERACTIVE HTML VISUALIZATION ===================================
# Advanced Pyvis-based interactive network visualization with legends and tooltips

def visualize_network(G: nx.DiGraph, output_filename: str, node_metrics: dict, edge_metrics: dict) -> bool:
    """
    Generates an interactive HTML file to visualize the network graph.
    
    Args:
        G (nx.DiGraph): The analyzed graph
        output_filename (str): Path to save the HTML file
        node_metrics (dict): Pre-calculated node size/color metrics
        edge_metrics (dict): Pre-calculated edge width/color metrics

    Returns:
        bool: True if successful, False otherwise
    """

    vis_config = CONFIG['visualization']
    pyvis_config = vis_config.get('pyvis_interactive', {}) # Get the new sub-config
    
    # Use fallback in case the new key is missing
    width = pyvis_config.get('width', '100%')
    height = pyvis_config.get('height', '90vh')
    notebook = pyvis_config.get('notebook', False)
    show_labels = pyvis_config.get("show_labels", True)
    show_tooltips = pyvis_config.get("show_tooltips", True)
    physics_solver = pyvis_config.get("physics_solver", "forceAtlas2Based")
    
    net = Network(
        height=height,
        width=width,
        directed=True,
        notebook=notebook,
        cdn_resources='remote',
        select_menu=True,
    )

    # Grey out nodes not connected to selected 
    net.highlight_nearest = True

    # The 'highlightNearest' object controls what happens to *unselected* items.
    options_json = {
        # Enable the GUI (equivalent to net.show_buttons)
        "configure": {
            "enabled": True, 
            "filter": ['physics'] # Only show the physics tab
        },
        
        # Custom 'physics' settings
        "physics": {
            "solver": physics_solver
        },

        "highlightNearest": {
            "enabled": True,
            "degree": 1, 
            "nodes": "all",
            "edges": "all", # Ensure edges are included in the dimming logic
            "unselectedColor": "#808080", # Grey color for unselected edges and nodes
            "unselectedNodeOpacity": 0.3, # Optional: Add opacity control
            "unselectedEdgeOpacity": 0.3, # Optional: Add opacity control
            "hover": True, # Keep edges highlighted on hover
            "hideWhenZooming": False # Keep dimming active when zooming
        },

        # Highlight selected edges 
        'interaction': {
            'hover': True,
            'hoverConnectedEdges': True,
            'selectConnectedEdges': True
        },
    }

    # Apply the custom options. This overrides the default highlighting behavior.
    net.set_options(json.dumps(options_json))

    print("--- Current net.options (after set_options):")
    # print(net.options) # Commented out for cleaner output
    print("------------------------------------------")

    # Add nodes using shared metrics
    for node, metrics in node_metrics.items():
        title_text = (
            f"{node}\n\n"
            f"Community ID: {metrics['community']}\n"
            f"Connections (Degree): {metrics['degree']}\n"
            f"Betweenness: {metrics['betweenness']:.4f}\n"
            f"Eigenvector (Influence): {metrics['eigenvector']:.4f}"
        )

        node_label = node if show_labels else None
        node_title = title_text if show_tooltips else None
        font_config = {'size': 0} if not show_labels else {}

        net.add_node(
            node,
            label=node_label,
            size=metrics['size'],
            color=metrics['color_hex'],
            title=node_title,
            font=font_config
        )
    
    # Add edges using shared metrics
    for (u, v), metrics in edge_metrics.items():
        title = (
            f"{u} <-> {v} (Mutual)\n" if metrics['is_mutual'] 
            else f"{u} -> {v} (One-way)\n"
        )
        title += f"Common Neighbors: {metrics['common_neighbors']}\n"
        title += (
            f"BRIDGE: Community {metrics['u_comm']} <-> {metrics['v_comm']}" 
            if metrics['is_bridge'] 
            else f"INTRA: Community {metrics['u_comm']}"
        )
        
        edge_title = title if show_tooltips else None
        dashes = not metrics['is_mutual']
        arrows = 'to, from' if metrics['is_mutual'] else 'to'

        net.add_edge(
            u, v, 
            title=edge_title,
            color=metrics['color'],
            width=metrics['width'],
            dashes=dashes,
            arrows=arrows
        )

    try:
        # Generate the HTML with legend
        html_string = net.generate_html()
        
        # Create legend HTML
        legend_html = create_html_legend(G, vis_config)
        
        # Insert legend into the HTML
        modified_html = insert_legend_into_html(html_string, legend_html)
        
        # Save the modified HTML
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(modified_html)
        
        print(f"SUCCESS: Interactive HTML with legend saved to: {output_filename}")
        return True
    except Exception as e:
        print(f"ERROR: Could not save interactive HTML: {e}")
        return False

def create_html_legend(G: nx.DiGraph, vis_config: dict) -> str:
    """
    Creates an HTML legend for the interactive visualization.
    
    Args:
        G (nx.DiGraph): Input graph with community attributes
        vis_config (dict): Visualization configuration
        
    Returns:
        str: HTML string for the legend
    """
    # Get community information
    communities_attr = nx.get_node_attributes(G, 'community')
    if communities_attr:
        num_communities = len(set(communities_attr.values()))
    else:
        # Fallback if analysis was skipped
        num_communities = 0 
        
    color_maps = get_community_colors(num_communities)
    color_map_hex = color_maps['hex']
    
    # Check if a metric was calculated or defaulted
    node_size_metric = vis_config['node_size_metric']
    if not communities_attr or node_size_metric == 'degree':
        # Default metric for degree, or if analysis was skipped (will default to actual degree)
        node_size_metric_display = 'Degree (Connections)' 
    else:
        node_size_metric_display = node_size_metric.capitalize()
    
    # Create community legend items
    community_items = ""
    if num_communities > 0:
        for i in range(min(num_communities, 10)):  # Limit to first 10 communities
            color = color_map_hex.get(i, '#808080')
            community_items += f'''
            <div style="display: flex; align-items: center; margin-bottom: 5px;">
                <div style="width: 20px; height: 20px; background-color: {color}; margin-right: 10px; border: 1px solid #ccc;"></div>
                <span>Community {i}</span>
            </div>
            '''
        
        if num_communities > 10:
            community_items += f'<div style="color: #666; font-style: italic;">...and {num_communities - 10} more communities</div>'
    else:
        community_items = '<div style="color: #666; font-style: italic;">(Community detection skipped or failed)</div>'
        
    
    legend_html = f'''
    <div id="legend" style="
        position: absolute; 
        bottom: 10px;
        right: 10px;
        background: rgba(255, 255, 255, 0.9); 
        border: 1px solid #ccc; 
        padding: 15px; 
        border-radius: 5px; 
        font-family: Arial, sans-serif; 
        font-size: 14px; 
        max-width: 300px;
        box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        z-index: 1000;
    ">
        <h3 style="margin-top: 0; margin-bottom: 15px; font-size: 16px; border-bottom: 1px solid #eee; padding-bottom: 5px;">Network Legend</h3>
        
        <div style="margin-bottom: 15px;">
            <h4 style="margin-bottom: 10px; font-size: 14px;">Communities</h4>
            {community_items}
        </div>
        
        <div style="margin-bottom: 15px;">
            <h4 style="margin-bottom: 10px; font-size: 14px;">Edge Types</h4>
            <div style="display: flex; align-items: center; margin-bottom: 5px;">
                <div style="width: 40px; height: 6px; background: #c0c0c0; margin-right: 10px;"></div>
                <span>Mutual Connection</span>
            </div>
            <div style="display: flex; align-items: center; margin-bottom: 5px;">
                <div style="width: 40px; height: 2px; background: #c0c0c0; margin-right: 10px; border: 1px dashed #666;"></div>
                <span>One-way Connection</span>
            </div>
            <div style="display: flex; align-items: center;">
                <div style="width: 40px; height: 2px; background: {vis_config.get('bridge_color', '#6e6e6e')}; margin-right: 10px;"></div>
                <span>Bridge Connection</span>
            </div>
        </div>
        
        <div style="margin-bottom: 15px;">
            <h4 style="margin-bottom: 10px; font-size: 14px;">Node Size</h4>
            <div>Larger nodes have higher {node_size_metric_display}</div>
        </div>
        
        <div style="font-size: 12px; color: #666;">
            <strong>Interactions:</strong><br>
            • Click and drag to move nodes<br>
            • Scroll to zoom<br>
            • Hover for details<br>
            • Drag background to pan
        </div>
    </div>
    '''
    
    return legend_html

def insert_legend_into_html(html_string: str, legend_html: str) -> str:
    """
    Inserts the legend HTML into the main HTML string.
    
    Args:
        html_string (str): Original HTML content
        legend_html (str): Legend HTML to insert
        
    Returns:
        str: Modified HTML with legend inserted
    """
    # Find the position to insert the legend (before the closing body tag)
    insert_position = html_string.find('</body>')
    
    if insert_position != -1:
        return html_string[:insert_position] + legend_html + html_string[insert_position:]
    else:
        # If no body tag found, append to the end
        return html_string + legend_html

# =================================== STATIC PNG VISUALIZATION ===================================
# Advanced matplotlib-based static network visualization with multiple layout options

def generate_static_graph(G: nx.DiGraph, output_filename: str, node_metrics: dict, edge_metrics: dict) -> bool:
    """
    Generates a static PNG image of the network graph using matplotlib.
    
    Args:
        G (nx.DiGraph): The analyzed graph
        output_filename (str): Path to save the PNG file
        node_metrics (dict): Pre-calculated node size/color metrics
        edge_metrics (dict): Pre-calculated edge width/color metrics
        
    Returns:
        bool: True if successful, False otherwise
    """
    static_config = CONFIG['visualization']['static_image']
    
    if G.number_of_nodes() == 0:
        print("WARNING: Cannot generate static graph. Graph is empty.")
        return False

    # Setup figure
    fig, ax = plt.subplots(figsize=static_config['image_size_inches'])
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    # Calculate layout
    layout_type = static_config['layout']
    G_undirected = G.to_undirected() 
    
    # Use ProgressTracker for spring layout, add timing for others
    if layout_type == 'spring':
        # Get parameters from config
        total_iterations = static_config.get('spring_iterations', 50)
        k_value = static_config.get('spring_k', 0.3)
        seed_value = 123
        
        # Using a fixed number of updates (e.g., 10 updates)
        num_updates = 10 
        chunk_size = max(1, total_iterations // num_updates)
        
        pos = None # Initialize position to None to start fresh
        
        # Use ProgressTracker
        title = f"Calculating 'spring' layout ({total_iterations} iterations)"
        tracker = ProgressTracker(total=total_iterations, title=title, num_updates=num_updates)
        
        # Loop through chunks
        for i in range(0, total_iterations, chunk_size):
            remaining_iters = total_iterations - i
            iters_in_this_step = min(chunk_size, remaining_iters)
            
            # Use 'pos' from the previous step to continue the layout calculation
            pos = nx.spring_layout(G_undirected, 
                                  k=k_value, 
                                  iterations=iters_in_this_step, 
                                  seed=seed_value, 
                                  pos=pos) 
            
            # Print progress update
            current_progress = i + iters_in_this_step
            tracker.update(current_progress)
            
        tracker.complete()

    else:
        # Add timing for other non-iterative layouts
        print(f"Calculating '{layout_type}' layout (this may take a while)...")
        start_layout_time = time.perf_counter()
        
        if layout_type == 'kamada_kawai':
            pos = nx.kamada_kawai_layout(G_undirected)
        elif layout_type == 'circular':
            pos = nx.circular_layout(G_undirected)
        elif layout_type == 'shell':
            communities_dict = nx.get_node_attributes(G, 'community')
            if communities_dict:
                num_communities = len(set(communities_dict.values()))
                nodelist = [[] for _ in range(num_communities)]
                for node, comm in communities_dict.items():
                    if comm < num_communities:
                        nodelist[comm].append(node)
                pos = nx.shell_layout(G, nodelist=nodelist)
            else:
                pos = nx.shell_layout(G_undirected)
        else:
            print(f"WARNING: Unknown layout '{layout_type}'. Defaulting to 'spring'.")
            # Use a single, non-iterative spring_layout as fallback
            pos = nx.spring_layout(G_undirected, seed=123)

        end_layout_time = time.perf_counter()
        print(f"    Layout finished in {end_layout_time - start_layout_time:.2f}s")

    # Prepare lists for batch drawing instead of drawing in a loop
    print("Preparing edge batches...")
    
    mutual_edges = []
    mutual_colors_rgb = []
    mutual_widths_mpl = []
    
    oneway_edges = []
    oneway_colors_rgb = []
    oneway_widths_mpl = []
    
    alpha = static_config.get('edge_alpha', 0.3)
    
    for (u, v), metrics in edge_metrics.items():
        # Convert hex color to RGB tuple
        hex_color = metrics['color'].lstrip('#')
        rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
        
        # Scale width for matplotlib (needs to be smaller)
        mpl_width = metrics['width'] * 0.8
        
        if metrics['is_mutual']:
            mutual_edges.append((u, v))
            mutual_colors_rgb.append(rgb_color)
            mutual_widths_mpl.append(mpl_width)
        else:
            oneway_edges.append((u, v))
            oneway_colors_rgb.append(rgb_color)
            oneway_widths_mpl.append(mpl_width)

    print(f"Drawing {len(mutual_edges):,} mutual edges...")
    nx.draw_networkx_edges(
        G, pos,
        edgelist=mutual_edges,
        width=mutual_widths_mpl,
        alpha=alpha,
        edge_color=mutual_colors_rgb,
        style='-',
        arrows=True,
        arrowsize=static_config.get('edge_arrow_size', 8),
        arrowstyle='<|-|>',
        ax=ax,
        node_size=0 # Prevent overlap issues
    )
    
    print(f"Drawing {len(oneway_edges):,} one-way edges...")
    nx.draw_networkx_edges(
        G, pos,
        edgelist=oneway_edges,
        width=oneway_widths_mpl,
        alpha=alpha,
        edge_color=oneway_colors_rgb,
        style='--',
        arrows=True,
        arrowsize=static_config.get('edge_arrow_size', 8),
        arrowstyle='-|>',
        ax=ax,
        node_size=0 # Prevent overlap issues
    )

    # Draw nodes
    print("Drawing nodes...")
    node_colors = [node_metrics[node]['color_rgba'] for node in G.nodes()]
    # Scale node sizes for matplotlib (multiply by large factor)
    node_sizes = [node_metrics[node]['size'] * 40 for node in G.nodes()]
    
    nx.draw_networkx_nodes(
        G, pos,
        node_color=node_colors,
        node_size=node_sizes,
        alpha=static_config.get('node_alpha', 0.8),
        ax=ax
    )

    # Add labels if requested
    if static_config['with_labels']:
        nx.draw_networkx_labels(
            G, pos,
            font_size=static_config['font_size'],
            font_color='black',
            ax=ax
        )

    # Add legend
    if static_config.get('show_legend', True):
        
        # Only show legend if community detection was run
        communities_attr = nx.get_node_attributes(G, 'community')
        if communities_attr:
            num_communities = len(set(communities_attr.values()))
            color_maps = get_community_colors(num_communities)
            
            legend_elements = [
                mpatches.Patch(facecolor=color_maps['rgba'][i], 
                               edgecolor='black', 
                               label=f'Community {i}')
                for i in range(min(num_communities, 10))  # Limit legend entries
            ]
        else:
            # Add a single default legend element if analysis skipped
            legend_elements = [
                mpatches.Patch(facecolor='#808080', 
                               edgecolor='black', 
                               label='Default/Unanalyzed')
            ]
        
        # Add edge type indicators
        legend_elements.extend([
            mpatches.Patch(facecolor='none', edgecolor='black', 
                           linestyle='-', label='Mutual Connection'),
            mpatches.Patch(facecolor='none', edgecolor='black', 
                           linestyle='--', label='One-way Connection')
        ])
        
        ax.legend(handles=legend_elements, loc='upper left', 
                  framealpha=0.9, fontsize=10)

    plt.axis('off')
    plt.tight_layout()
    
    # Save the figure
    try:
        plt.savefig(output_filename, dpi=static_config['dpi'], bbox_inches='tight')
        print(f"SUCCESS: Static PNG saved to: {output_filename}")
        return True
    except Exception as e:
        print(f"ERROR: Could not save static PNG: {e}")
        return False
    finally:
        plt.close()

# =================================== METRICS REPORTING AND TEXT OUTPUT GENERATION ===================================
# Comprehensive metrics reporting with analysis parameters and timing information

def save_metrics_report(G: nx.DiGraph, filename: str, config: dict, timing_info: dict = None) -> bool:
    """
    Saves a detailed text report with network statistics, analysis parameters, and timing information.
    
    Args:
        G (nx.DiGraph): The analyzed graph
        filename (str): Path to save the text report
        config (dict): Configuration dictionary used for analysis
        timing_info (dict): Optional timing information for different analysis steps
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            # Header
            f.write("=" * 80 + "\n")
            f.write("FOLLOWWEB NETWORK ANALYSIS REPORT\n")
            f.write("=" * 80 + "\n")
            f.write(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Analysis Strategy: {config['pipeline']['strategy']}\n")
            f.write(f"Input File: {config['input_file']}\n")
            f.write("\n")
            
            # Configuration Summary
            f.write("CONFIGURATION SUMMARY\n")
            f.write("-" * 40 + "\n")
            f.write(f"Strategy: {config['pipeline']['strategy']}\n")
            f.write(f"Skip Analysis: {config['pipeline']['skip_analysis']}\n")
            
            if config['pipeline']['strategy'] == 'ego_alter_k-core':
                f.write(f"Ego Username: {config['pipeline']['ego_username']}\n")
            
            strategy = config['pipeline']['strategy']
            k_value = config['pruning']['k_values'].get(strategy, config['pruning']['default_k_value'])
            f.write(f"K-value (pruning threshold): {k_value}\n")
            
            f.write(f"Node Size Metric: {config['visualization']['node_size_metric']}\n")
            f.write(f"Scaling Algorithm: {config['visualization']['scaling_algorithm']}\n")
            f.write("\n")
            
            # Fame Analysis Configuration
            f.write("FAME ANALYSIS CONFIGURATION\n")
            f.write("-" * 40 + "\n")
            f.write(f"Find Paths to Famous: {config['fame_analysis']['find_paths_to_all_famous']}\n")
            f.write(f"Min Followers in Network: {config['fame_analysis']['min_followers_in_network']}\n")
            f.write(f"Min Fame Ratio: {config['fame_analysis']['min_fame_ratio']}\n")
            f.write("\n")
            
            # Network Statistics
            f.write("NETWORK STATISTICS\n")
            f.write("-" * 40 + "\n")
            f.write(f"Total Nodes: {G.number_of_nodes():,}\n")
            f.write(f"Total Edges: {G.number_of_edges():,}\n")
            
            if G.number_of_nodes() > 0:
                # Basic network metrics
                density = nx.density(G)
                f.write(f"Network Density: {density:.6f}\n")
                
                # Degree statistics
                degrees = dict(G.degree())
                if degrees:
                    degree_values = list(degrees.values())
                    f.write(f"Average Degree: {sum(degree_values) / len(degree_values):.2f}\n")
                    f.write(f"Max Degree: {max(degree_values)}\n")
                    f.write(f"Min Degree: {min(degree_values)}\n")
                
                # Community information
                communities_attr = nx.get_node_attributes(G, 'community')
                if communities_attr:
                    num_communities = len(set(communities_attr.values()))
                    f.write(f"Number of Communities: {num_communities}\n")
                    
                    # Community size distribution
                    community_sizes = {}
                    for node, comm in communities_attr.items():
                        community_sizes[comm] = community_sizes.get(comm, 0) + 1
                    
                    if community_sizes:
                        sizes = list(community_sizes.values())
                        f.write(f"Average Community Size: {sum(sizes) / len(sizes):.2f}\n")
                        f.write(f"Largest Community Size: {max(sizes)}\n")
                        f.write(f"Smallest Community Size: {min(sizes)}\n")
                else:
                    f.write("Community Detection: Skipped or Failed\n")
                
                # Centrality information
                betweenness_attr = nx.get_node_attributes(G, 'betweenness')
                eigenvector_attr = nx.get_node_attributes(G, 'eigenvector')
                
                if betweenness_attr:
                    bet_values = list(betweenness_attr.values())
                    f.write(f"Average Betweenness Centrality: {sum(bet_values) / len(bet_values):.6f}\n")
                    f.write(f"Max Betweenness Centrality: {max(bet_values):.6f}\n")
                
                if eigenvector_attr:
                    eig_values = list(eigenvector_attr.values())
                    f.write(f"Average Eigenvector Centrality: {sum(eig_values) / len(eig_values):.6f}\n")
                    f.write(f"Max Eigenvector Centrality: {max(eig_values):.6f}\n")
                
                f.write("\n")
                
                # Top nodes by different metrics
                f.write("TOP NODES BY METRICS\n")
                f.write("-" * 40 + "\n")
                
                # Top by degree
                top_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:10]
                f.write("Top 10 Nodes by Degree:\n")
                for i, (node, degree) in enumerate(top_degree, 1):
                    f.write(f"  {i:2d}. {node:<20} | Degree: {degree}\n")
                f.write("\n")
                
                # Top by betweenness if available
                if betweenness_attr:
                    top_betweenness = sorted(betweenness_attr.items(), key=lambda x: x[1], reverse=True)[:10]
                    f.write("Top 10 Nodes by Betweenness Centrality:\n")
                    for i, (node, centrality) in enumerate(top_betweenness, 1):
                        f.write(f"  {i:2d}. {node:<20} | Betweenness: {centrality:.6f}\n")
                    f.write("\n")
                
                # Top by eigenvector if available
                if eigenvector_attr:
                    top_eigenvector = sorted(eigenvector_attr.items(), key=lambda x: x[1], reverse=True)[:10]
                    f.write("Top 10 Nodes by Eigenvector Centrality:\n")
                    for i, (node, centrality) in enumerate(top_eigenvector, 1):
                        f.write(f"  {i:2d}. {node:<20} | Eigenvector: {centrality:.6f}\n")
                    f.write("\n")
            
            # Timing Information
            if timing_info:
                f.write("PERFORMANCE TIMING\n")
                f.write("-" * 40 + "\n")
                for step_name, duration in timing_info.items():
                    f.write(f"{step_name}: {duration:.4f} seconds\n")
                f.write("\n")
            
            # Analysis Parameters
            f.write("DETAILED CONFIGURATION\n")
            f.write("-" * 40 + "\n")
            f.write("Pipeline Configuration:\n")
            for key, value in config['pipeline'].items():
                f.write(f"  {key}: {value}\n")
            f.write("\n")
            
            f.write("Pruning Configuration:\n")
            for key, value in config['pruning'].items():
                f.write(f"  {key}: {value}\n")
            f.write("\n")
            
            f.write("Visualization Configuration:\n")
            for key, value in config['visualization'].items():
                if isinstance(value, dict):
                    f.write(f"  {key}:\n")
                    for sub_key, sub_value in value.items():
                        f.write(f"    {sub_key}: {sub_value}\n")
                else:
                    f.write(f"  {key}: {value}\n")
            f.write("\n")
            
            # Footer
            f.write("=" * 80 + "\n")
            f.write("END OF REPORT\n")
            f.write("=" * 80 + "\n")
        
        print(f"SUCCESS: Metrics report saved to: {filename}")
        return True
        
    except Exception as e:
        print(f"ERROR: Could not save metrics report: {e}")
        return False

def generate_text_output_filename(prefix: str, strategy: str, k_value: int) -> str:
    """
    Generates a text report filename with the same base as visualizations.
    
    Args:
        prefix (str): Base filename prefix
        strategy (str): Analysis strategy name
        k_value (int): K-value used for pruning
        
    Returns:
        str: Text report filename
    """
    return generate_output_filename(prefix, strategy, k_value, 'txt')


# =================================== ENHANCED VISUALIZATION GENERATION ===================================
# Generate both interactive HTML and static PNG visualizations with advanced styling

def numEdges(node_id: str) -> int:
    """
    Helper function to get the number of edges for a node (for legacy compatibility).
    
    Args:
        node_id (str): Node identifier
        
    Returns:
        int: Number of edges (degree) for the node
    """
    if node_id in G:
        return G.degree(node_id)
    return 0

# Calculate node and edge metrics for visualization
print("=== CALCULATING VISUALIZATION METRICS ===")
viz_metrics_start = time.perf_counter()
log_analysis_step("Visualization Metrics Started", "Calculating node and edge properties")

node_metrics = calculate_node_metrics(G, CONFIG['visualization'])
edge_metrics = calculate_edge_metrics(G, CONFIG['visualization'])

viz_metrics_end = time.perf_counter()
timing_info['Visualization Metrics'] = viz_metrics_end - viz_metrics_start
log_analysis_step("Visualization Metrics Complete", 
                 f"{len(node_metrics)} nodes, {len(edge_metrics)} edges", 
                 timing_info['Visualization Metrics'])

print(f"Calculated metrics for {len(node_metrics)} nodes and {len(edge_metrics)} edges.")
print("=============================================\n")

# =================================== GENERATE INTERACTIVE HTML VISUALIZATION ===================================
# Create advanced interactive network visualization with legends and tooltips

if G.number_of_nodes() > 0:
    print("=== GENERATING INTERACTIVE HTML VISUALIZATION ===")
    html_gen_start = time.perf_counter()
    
    # Generate output filename
    strategy = CONFIG['pipeline']['strategy']
    k_value = CONFIG['pruning']['k_values'].get(strategy, CONFIG['pruning']['default_k_value'])
    html_filename = generate_output_filename(
        CONFIG['output_file_prefix'], 
        strategy, 
        k_value, 
        'html'
    )
    
    # Ensure output directory exists
    ensure_output_directory(html_filename)
    log_analysis_step("HTML Generation Started", f"Output: {html_filename}")
    
    # Generate interactive HTML visualization
    success = visualize_network(G, html_filename, node_metrics, edge_metrics)
    
    html_gen_end = time.perf_counter()
    timing_info['HTML Generation'] = html_gen_end - html_gen_start
    
    if success:
        log_analysis_step("HTML Generation Complete", 
                         f"Interactive visualization saved", 
                         timing_info['HTML Generation'])
        print(f"Interactive HTML visualization saved to: {html_filename}")
    else:
        log_analysis_step("HTML Generation Failed", "Could not create interactive visualization")
        print("Failed to generate interactive HTML visualization")
        
    print("================================================\n")
else:
    print("=== SKIPPING VISUALIZATION (empty graph) ===")
    log_analysis_step("HTML Generation Skipped", "Empty graph")
    timing_info['HTML Generation'] = 0.0
    print("============================================\n")

# =================================== GENERATE STATIC PNG VISUALIZATION ===================================
# Create high-resolution static network visualization with multiple layout options

if G.number_of_nodes() > 0 and CONFIG['visualization']['static_image']['generate']:
    print("=== GENERATING STATIC PNG VISUALIZATION ===")
    png_gen_start = time.perf_counter()
    
    # Generate output filename
    strategy = CONFIG['pipeline']['strategy']
    k_value = CONFIG['pruning']['k_values'].get(strategy, CONFIG['pruning']['default_k_value'])
    png_filename = generate_output_filename(
        CONFIG['output_file_prefix'], 
        strategy, 
        k_value, 
        'png'
    )
    
    # Ensure output directory exists
    ensure_output_directory(png_filename)
    log_analysis_step("PNG Generation Started", f"Output: {png_filename}")
    
    # Generate static PNG visualization
    success = generate_static_graph(G, png_filename, node_metrics, edge_metrics)
    
    png_gen_end = time.perf_counter()
    timing_info['PNG Generation'] = png_gen_end - png_gen_start
    
    if success:
        log_analysis_step("PNG Generation Complete", 
                         f"Static visualization saved", 
                         timing_info['PNG Generation'])
        print(f"Static PNG visualization saved to: {png_filename}")
    else:
        log_analysis_step("PNG Generation Failed", "Could not create static visualization")
        print("Failed to generate static PNG visualization")
        
    print("===============================================\n")
elif G.number_of_nodes() == 0:
    print("=== SKIPPING STATIC PNG (empty graph) ===")
    log_analysis_step("PNG Generation Skipped", "Empty graph")
    timing_info['PNG Generation'] = 0.0
    print("=========================================\n")
else:
    print("=== SKIPPING STATIC PNG (disabled in config) ===")
    log_analysis_step("PNG Generation Skipped", "Disabled in configuration")
    timing_info['PNG Generation'] = 0.0
    print("================================================\n")

# =================================== GENERATE METRICS REPORT ===================================
# Create detailed text report with network statistics and analysis parameters

if G.number_of_nodes() > 0:
    print("=== GENERATING METRICS REPORT ===")
    metrics_report_start = time.perf_counter()
    
    # Generate output filename for metrics report
    strategy = CONFIG['pipeline']['strategy']
    k_value = CONFIG['pruning']['k_values'].get(strategy, CONFIG['pruning']['default_k_value'])
    txt_filename = generate_text_output_filename(
        CONFIG['output_file_prefix'], 
        strategy, 
        k_value
    )
    
    # Ensure output directory exists
    ensure_output_directory(txt_filename)
    log_analysis_step("Metrics Report Started", f"Output: {txt_filename}")
    
    # Generate metrics report with actual timing information
    success = save_metrics_report(G, txt_filename, CONFIG, timing_info)
    
    metrics_report_end = time.perf_counter()
    timing_info['Metrics Report'] = metrics_report_end - metrics_report_start
    
    if success:
        log_analysis_step("Metrics Report Complete", 
                         f"Detailed report saved", 
                         timing_info['Metrics Report'])
        print(f"Metrics report saved to: {txt_filename}")
    else:
        log_analysis_step("Metrics Report Failed", "Could not create report")
        print("Failed to generate metrics report")
        
    print("=====================================\n")
else:
    print("=== SKIPPING METRICS REPORT (empty graph) ===")
    log_analysis_step("Metrics Report Skipped", "Empty graph")
    timing_info['Metrics Report'] = 0.0
    print("==============================================\n")

# =================================== PIPELINE COMPLETION AND SUMMARY ===================================
# Log comprehensive analysis summary and final timing information

# Calculate total pipeline time
pipeline_end_time = time.perf_counter()
total_pipeline_time = pipeline_end_time - pipeline_start_time
timing_info['Total Pipeline'] = total_pipeline_time

# Log final summary
log_analysis_step("Pipeline Complete", f"Total execution time: {total_pipeline_time:.4f}s")
log_analysis_summary(G, CONFIG, timing_info)

# =================================== LEGACY COMPATIBILITY ===================================
# Initialize Pyvis network for downstream compatibility
net = Network(700, 700, directed=True, notebook=False)  # For jupyter notebook = True

#======================================== NETWORKX TO PYVIS =============================================
#Aesthetic Options
sizeByConnections = 1 #Change a nodes size by number of connections 

# Add nodes and edges from the processed NetworkX graph to the Pyvis network
net.from_nx(G)

# Apply size scaling if enabled
if sizeByConnections:
    for node in net.nodes:
        node['size'] = (numEdges(node['id'])/50)+9

# Set physics options for the visualization
net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.07, damping=0.8, overlap=1)
net.show_buttons(filter_=['physics'])

# Generate and show the HTML file
net.save_graph("FollowWeb.html")
print("Legacy visualization complete. Check 'FollowWeb.html'.")
